Conexión por SSH
==

In [24]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "gonzalo"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

 ··········


Ejecución de comandos
==

Implementé un *servidor MCLP*, el cual escucha instrucciones para generar bins.

El siguiente código lanza el servidor en el background, carga una instancia y lo deja escuchando peticiones por el puerto 8080

In [26]:
transport = ssh.get_transport()
channel = transport.open_session()
channel.exec_command("./mclp/Metasolver/BASIC_BSG_MCLP mclp/Metasolver/problems/mclp/benchs/class1/100.txt -i 2")

**Generación de conjunto inicial de bins con BSG**

La siguiente función genera la lista inicial de bins. Cada línea representa un bin indicando el porcentaje de llenado y los ids de sus cajas.

Es necesario modificarla para que en vez de imprimir el contenido de los bins, retorne una estructura adecuada.

In [27]:
def generate_bins():
    stdin, stdout, stderr = ssh.exec_command("echo generate_bins | netcat localhost 8080")
    flag = False
    for line in stdout.readlines():
        if "BINS:" in line: 
            flag = True
            continue
        if flag:
            print(line.strip())
generate_bins()

0.6495 0 8 45 53
0.858 1 12 28 78 85 93
0.7557 2 61 75
0.7324 3 19 79
0.9444 4 41 60 63 82
0.7073 5 47 59 94
0.8826 6 11 18 64 84
0.5461 7 36 57
0.7696 9 42 92 99
0.314 10
0.7782 13 77
0.4622 14 43
0.935 15 32 40 51 72 73
0.8123 16 81 90
0.8995 17 20 23 38 46 97
0.5697 21 22
0.8657 24 30 33 34 39 55
0.6572 25 29 54
0.8497 26 37 88
0.4925 27 96
0.7205 31 71 91
0.6375 35 68 80
0.8015 44 65 66
0.8811 48 50 70 76
0.8034 49 67 69 83
0.5525 52
0.194 56
0.5859 58 74
0.7643 62 95 98
0.7842 86 87 89


**Construcción de bin priorizando cajas (BSG)**

In [21]:
def generate_bin(boxes, priority_boxes):
    boxes_str = [str(box) for box in boxes]
    pboxes_str = [str(pbox) for pbox in priority_boxes]    
    stdin, stdout, stderr = ssh.exec_command("echo generate_bin " + " ".join(boxes_str) + " -1 " + " ".join(pboxes_str) +  " -2 | netcat localhost 8080")
    for line in stdout.readlines(): print(line.strip())
        
generate_bin([0,1,8,53,45],[45])

0.6659 0 1 45
8 53


Para cerrar el servidor MCLP

In [ ]:
def close_mclp_server():
    stdin, stdout, stderr = ssh.exec_command("echo END | netcat localhost 8080")
    print(stdout.readlines()[0])
    
close_mclp_server()

Faltaría:

* Cargar la instancia en python (cajas con sus dimensiones)
* Implementar algoritmo.

A grandes rasgos el algoritmo hace los siguiente:
1. Generación de bins iniciales usando **BSG**
2. Selección de bin a desarmar y almacenar cajas en $C$
3. Mientras $C$ no quede vacío o máximo de iteraciones:

   1. Seleccionar caja $c$ de $C$
   2. Seleccionar bin de destino $B$
   3. Usar **BSG** para generar bin $B'$ usando cajas $B \cup \{c\}$, priorizando $c$. Es posible que **BSG** retorne conjunto de cajas residuales $R$
   4. Si $R$ es mejor que $c$, $B$ se reemplaza por $B'$ en el conjunto de bins y $C \gets C \cup R$
   
 4. Volver a 2 (seleccionar otro bin para desarmar)